In [5]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
#base url
base_url = 'https://fantasy.premierleague.com/api/'

r = requests.get(base_url +'bootstrap-static/').json()

In [9]:
# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10997846}


# Player data

The elements field contains data for each premier league player in the current season of FPL. We can access these data just like we would access data associated with a key in a dictionary. The response is a list of more dictionaries — one per player.

Let’s get the elements data and then show the information about the first player in the list:

In [13]:
players = r['elements']
pprint((players[4]))

In [16]:
pd.set_option('display.max_columns', None)

#create players dataframe 
players = pd.json_normalize(r['elements'])

In [18]:
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Cédric,1,2
1,3,Xhaka,1,3
2,4,Elneny,1,3
3,5,Holding,1,2
4,6,Partey,1,3


# Supporting data 

We can get the names and strength ratings for teams by extracting the teams field from the base response into a dataframe:

In [19]:
#create teams dataframe 

teams = pd.json_normalize(r['teams'])

In [20]:
teams

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1060,1090,1070,1130,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1100,1130,1100,1110,1130,1160,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1150,1160,1140,1160,1170,1190,131
5,8,0,None,6,0,Chelsea,0,0,0,CHE,4,None,False,0,1200,1220,1200,1210,1180,1210,4
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,3,None,False,0,1100,1140,1100,1170,1080,1140,6
7,11,0,None,8,0,Everton,0,0,0,EVE,3,None,False,0,1080,1100,1080,1080,1080,1080,7
8,54,0,None,9,0,Fulham,0,0,0,FUL,3,None,False,0,1075,1100,1070,1070,1100,1130,34
9,13,0,None,10,0,Leicester,0,0,0,LEI,3,None,False,0,1130,1130,1080,1160,1180,1120,26
